In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
print(tf.__version__)

2.11.0


In [ ]:
mnist = keras.datasets.mnist
(train_images_mnist,train_labels_mnist),(test_images_mnist,test_labels_mnist) = mnist.load_data()
# images are reshaped to be used by the flow method of a keras ImageGenerator
train_images_mnist = np.reshape(train_images_mnist,(train_images_mnist.shape[0],28,28,1))  
test_images_mnist = np.reshape(test_images_mnist,(test_images_mnist.shape[0],28,28,1))

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
az_data_path = '/content/drive/MyDrive/A_Z Handwritten Data.csv'
AZ_data = pd.read_csv(az_data_path, header = None)
# the first column contains label values, while the remaining are the flattened array of 28 x 28 image pixels
AZ_labels = AZ_data.values[:,0]
AZ_images = AZ_data.values[:,1:]
# images are reshaped to be used by the flow method of a keras ImageGenerator
AZ_images = np.reshape(AZ_images,(AZ_images.shape[0],28,28,1))  

In [ ]:
# join datasets
# split AZ data in train and test
from sklearn.model_selection import train_test_split

test_size = float(len(test_labels_mnist))/len(train_labels_mnist)
print(f'test set size: {test_size}')
train_images_AZ, test_images_AZ, train_labels_AZ, test_labels_AZ = train_test_split(AZ_images,AZ_labels, test_size=test_size)
#shift mnist labels 
train_labels_mnist = train_labels_mnist + max(AZ_labels)+1
test_labels_mnist = test_labels_mnist + max(AZ_labels)+1

# concatenate datasets
train_images = np.concatenate((train_images_AZ,train_images_mnist),axis=0)
train_labels = np.concatenate((train_labels_AZ,train_labels_mnist))
test_images = np.concatenate((test_images_AZ,test_images_mnist),axis=0)
test_labels = np.concatenate((test_labels_AZ,test_labels_mnist))

print('Data ready')

test set size: 0.16666666666666666
Data ready


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(len(np.unique(train_labels)), activation='softmax')  
])

model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 512)               4

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=15,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Flow training images in batches using generator
train_generator = train_datagen.flow(train_images, train_labels, batch_size=50, shuffle=True)
validation_generator = test_datagen.flow(test_images, test_labels, batch_size=50, shuffle=True)

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=500,  
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50,  
      verbose=2)
model.save('model_v2')

Epoch 1/50
500/500 - 34s - loss: 2.5567 - accuracy: 0.3266 - val_loss: 1.3520 - val_accuracy: 0.6312 - 34s/epoch - 68ms/step
Epoch 2/50
500/500 - 33s - loss: 1.6621 - accuracy: 0.5466 - val_loss: 0.7945 - val_accuracy: 0.7912 - 33s/epoch - 66ms/step
Epoch 3/50
500/500 - 32s - loss: 1.2511 - accuracy: 0.6536 - val_loss: 0.5858 - val_accuracy: 0.8356 - 32s/epoch - 64ms/step
Epoch 4/50
500/500 - 33s - loss: 1.0350 - accuracy: 0.7100 - val_loss: 0.4412 - val_accuracy: 0.8792 - 33s/epoch - 66ms/step
Epoch 5/50
500/500 - 35s - loss: 0.8670 - accuracy: 0.7582 - val_loss: 0.3905 - val_accuracy: 0.8884 - 35s/epoch - 71ms/step
Epoch 6/50
500/500 - 31s - loss: 0.7659 - accuracy: 0.7844 - val_loss: 0.3616 - val_accuracy: 0.9076 - 31s/epoch - 63ms/step
Epoch 7/50
500/500 - 32s - loss: 0.6822 - accuracy: 0.8068 - val_loss: 0.2922 - val_accuracy: 0.9196 - 32s/epoch - 64ms/step
Epoch 8/50
500/500 - 33s - loss: 0.6199 - accuracy: 0.8212 - val_loss: 0.3067 - val_accuracy: 0.9188 - 33s/epoch - 66ms/step
